In [1]:
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2

#import opencv as cv2

img_array = np.load('test_images.npy',encoding='latin1')

images = np.load('train_images.npy',encoding='latin1')

In [2]:
import csv
def readCSV(path):
    with open(path) as csv_file:
        result = []
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            line = []
            for data in row:
                line.append(data)
            result.append(line)
    return result

In [3]:
input=[]
for i in range(len(images)):
    image1 = (images[i][1]).reshape(100,100)
    #plt.imshow(image1)
    img = cv2.imwrite('temp.jpg',image1)
    img = cv2.imread('temp.jpg',0)
    edges = cv2.Canny(img, 0, 100)
    #plt.imshow(edges)

    im2, contours, hierarchy = cv2.findContours(edges, cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(contours, key = cv2.contourArea, reverse = True)[:1]


    mask=np.zeros(img.shape)
    cv2.drawContours(mask, cnts, -1, (255),1)
    input.append(mask)
    #plt.imshow(mask)


In [4]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import ShuffleSplit
labels = np.array(readCSV('train_labels.csv'))#np.loadtxt('train_labels.csv',dtype=str,delimiter=',')
X = [x.reshape(10000,) for x in input]
y = labels[1:,1].tolist()

# print(y)

C:\Users\judyy\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [10]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encoder.fit(y)
y_encoded = encoder.transform(y)
print(set(y_encoded))
# y = np.loadtxt(label_data, dtype=str, encoding='latin1', delimiter=',')[1:,1]

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}


In [203]:
import numpy as np
import skimage.measure
x = [skimage.measure.block_reduce(a, (4,4), np.max) for a in input]
print(len(x[0]))
# X = [a.reshape(625,1) for a in x]

25


In [209]:

# train = []
# for i in range(len(X)):
#     a = []
#     for k in X[i]:
#         a.append([k/255])
#     train.append([a,[y_encoded[i]]])
# print(train[0])

data_y = [preprocess_y(y) for y in y_encoded]
data_x = [a.reshape((625,1)) for a in x]
full_data = list(zip(data_x, data_y))
random.shuffle(full_data)
print(full_data[0])

(array([[   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       

In [214]:
#do an 80/20 split for training/test: 
num_examples = len(full_data)
split = int(0.8 * num_examples)
train_data = full_data[:split]
test_data = full_data[split:]

## forward propagation

In [217]:
import random

def preprocess_y(y):
    y = int(y)
    new_y = np.zeros(31)
    new_y[y] = 1.0
    return new_y.T.reshape((31,1))


def sigmoid(z, derivative):
    sig = 1.0 / (1.0 + np.exp(-z))
    if derivative == False:
        return sig
    else:
        return sig * (1 - sig)   

class Network(object):
    def __init__(self, sizes, norm, mom, lr):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.__init__weights()
        self.__init__biases()
        self.momentum_coef = mom
        self.norm = norm
        self.lr = lr
        
    def __init__weights(self):
        #weights are initialized using Glorot initialization
        self.weights = [np.random.uniform(-np.sqrt(6)/np.sqrt(x+y), np.sqrt(6)/np.sqrt(x+y),(y, x)) for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def __init__biases(self):
        #Biases are initialized randomly
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
  
    #Feed Forward method computes the output vector of Neural Network assuming that the weights are trained
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = np.dot(w, a) + b
            a = sigmoid(a,derivative = False)
        return a
    
    def Stochasitc_Gradient_Descent(self, training_data, epochs, mini_batch_size, test_data = None):
        learning_rate = self.lr
        best_result = 0
        n = len(training_data)
        if test_data != None: 
            n_test = len(test_data)
        test_results = [0.0]    
        for j in range(epochs):
            random.shuffle(training_data)
            batches = [training_data[k : k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            #Momentum Terms used to speed up crossing saddle points
            gradient_biases_old = [np.zeros(b.shape) for b in self.biases]
            gradient_weights_old = [np.zeros(w.shape) for w in self.weights]
            
            for batch in batches:
                gradient_biases = [np.zeros(b.shape) for b in self.biases]
                gradient_weights = [np.zeros(w.shape) for w in self.weights]
                
                #Momentum addition of previous gradient values
                momentum_biases = np.multiply(self.momentum_coef,gradient_biases_old)
                momentum_weights = np.multiply(self.momentum_coef,gradient_weights_old)
                
                #compute dC/dW for each example in batch and add them to gradent arrays
                for x, y in batch:
                    d_gradient_biases, d_gradient_weights = self.Compute_Gradient_Term(x, y)
                    gradent_biases = [gb + dgb for gb, dgb in zip(gradient_biases, d_gradient_biases)]
                    gradient_weights = [gw + dgw for gw, dgw in zip(gradient_weights, d_gradient_weights)]

                #Standard terms of the gradient update
                prefactor = float(-learning_rate / len(batch))
                gradient_biases  = np.multiply(prefactor, gradient_biases)
                gradient_weights  = np.multiply(prefactor, gradient_weights)
                
                #Update previous gradient updates to current gradients
                gradient_biases_old = gradient_biases
                gradient_weights_old = gradient_weights
                
                #Add update to weights and continue on next mini-batch
                self.weights = [(1-((self.norm*learning_rate)/len(batch)))*w + gw + mw  for w, gw, mw in zip(self.weights,gradient_weights, momentum_weights)]
                self.biases = [b + gb + mb for b, gb, mb in zip(self.biases,gradient_biases, momentum_biases)]

            
            if test_data:
                test = self.evaluate(test_data)
                print ("Epoch %d: %f" %(j+1, test/float(n_test)))
                test_results.append(test)
                #Adaptive Learning Rate:
                result_improvement = np.abs(test_results[-1] - test_results[-2])/ float(test_results[-2])
                #if there is little improvement from previous to current result then halve the learning rate
                if result_improvement < 0.001:
                    learning_rate = learning_rate / 2
                    #end the reduction of the learning rate
                    if learning_rate < 0.0005:
                        learning_rate = 0.0005
            else:
                print ("Epoch %d complete" %(j+1))
        print ("Training Completed!")
        return test_results

    def Compute_Gradient_Term(self, x, y):
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        #Feed Forward Step: pass the input into the neural network with the current version of the weights
        #and biases in order to obtain both the outputs at each layer and the activations at each layer
        activations = [x] 
        outputs = [] 
        for b, w in zip(self.biases, self.weights):
#             print('w')
#             print(w)
#             print('x')
#             print(x.shape)
#             print('dot')
#             print(np.dot(w,x))
#             print('b')
#             print(b)
            x = np.dot(w, x)+b
            outputs.append(x)
#             print('output')
#             print(x)
            x = sigmoid(x, derivative = False)
            activations.append(x)
#             print('act')
#             print(x)
        #Backward Pass: back propogate the errors of the previous forward pass step in order to compute
        #the gradient updates for this specific image in the mini-batch
        #The gradients of biases and weights will be averaged across the mini-batch and the update will be applied.
#         print(activations[-1] - y)
#         print(activations[-1])
        
        delta = (activations[-1] - y) * sigmoid(outputs[-1], derivative = True)
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].T)
        for n in range(2, self.num_layers):
            delta = np.dot(self.weights[-n+1].T, delta) * sigmoid(outputs[-n], derivative = True)
            grad_b[-n] = delta
            grad_w[-n] = np.dot(delta, np.transpose(activations[-n-1]))
        return (grad_b, grad_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in test_data]
        results = sum(int(x == y) for (x, y) in test_results)
        return results
    
    def save_values(self):
        np.savetxt("../best_weights_0.txt",self.weights[0])
        np.savetxt("../best_weights_1.txt",self.weights[1])
        np.savetxt("../best_biases_0.txt",self.biases[0])
        np.savetxt("../best_biases_1.txt",self.biases[1])

In [223]:
print ("Training Network 1 on %d preprocessed training images" % len(train_data))
shape_1 = [625, 300, 31]
net_1 = Network(shape_1, 0.1, 0.1, 0.1)
ModMNIST_results = net_1.Stochasitc_Gradient_Descent(train_data, 30, 1000, test_data) 


Training Network 1 on 8000 preprocessed training images
Epoch 1: 0.027500


C:\Users\judyy\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in double_scalars


Epoch 2: 0.029000
Epoch 3: 0.031000
Epoch 4: 0.033000
Epoch 5: 0.036500
Epoch 6: 0.038500
Epoch 7: 0.039000
Epoch 8: 0.040000
Epoch 9: 0.041500
Epoch 10: 0.041500
Epoch 11: 0.041500
Epoch 12: 0.042000
Epoch 13: 0.041000
Epoch 14: 0.041000
Epoch 15: 0.041000
Epoch 16: 0.041000
Epoch 17: 0.040500
Epoch 18: 0.040500
Epoch 19: 0.040500
Epoch 20: 0.040500
Epoch 21: 0.040500
Epoch 22: 0.040500
Epoch 23: 0.040500
Epoch 24: 0.040500
Epoch 25: 0.040500
Epoch 26: 0.040500
Epoch 27: 0.040500
Epoch 28: 0.040500
Epoch 29: 0.040500
Epoch 30: 0.040500
Training Completed!
